# Progetto AADS 
## Analisi sul numero di nodi esterni (foglie) dato in numero di nodi interni

#### Autore: Lorenzo Pratesi Mariti -  - <a href="mailto:lorenzo.pratesi@stud.unifi.it">lorenzo.pratesi@stud.unifi.it</a>

##### Descrizione del progetto:
In questo notebook verranno effettuate verifiche numeriche sul numero medio di foglie nelle strutture dati ad albero. 

### Preparazione:
Per prima cosa è necessario importare tutti i pacchetti necessari allo svolgimento di questo progetto:

In [1]:
from tqdm import tqdm
from sage.combinat.species.library import *
from sympy import init_printing
init_printing()

In [2]:
from lib.sage_itertools import *
from lib.backtraking import *
from lib.mary_tree import *

## Alberi $s$-ari
Un albero $s$-ario è una struttura definita in modo ricorsivo come un nodo esterno oppure un nodo interno che è connesso ad $s$ alberi $s$-ari. Se si indica con $\mathcal{T}$ la classe degli alberi $s$-ari, questa definizione si traduce nella seguente relazione:

$$\mathcal{T} = \{•\} + \{◦\} × \mathcal{T} × \mathcal{T} · · · × \mathcal{T}$$

Applicando il metodo simbolico, si ha che la funzione generatrice che conta il numero di alberi $s$-ari con $n$ nodi interni soddisfa l'equazione: ${T}(t) = 1 + {T}(t)^s$
Per l’estrazione del coefficiente si procede come segue:

$$T(t) − 1 = tT(t)^s,\quad T_0 = 1$$

$$A(t) = t(1 + A(t))^s,\quad A(t) = T(t) − 1$$

$$T_n = A_n =\frac{1}{n}[u^{n−1}](1 + u)^{sn} =\frac{1}{n}\binom{sn}{n − 1}=\frac{1}{(s − 1)n + 1}\binom{sn}{n}.$$

### Verifica del risultato del numero di alberi $s$-ari dato il numero di nodi interni.

In [3]:
# 5-ary trees
s = 5

In [4]:
o = var('o')
N = SingletonSpecies(); 
L = EmptySetSpecies()
TreeS = CombinatorialSpecies()

TreeS.define(L + N * TreeS ^ s)
TreeS.isotypes([o] * s).cardinality()
TreeS.isotype_generating_series()

1 + x + 5*x^2 + 35*x^3 + 285*x^4 + 2530*x^5 + O(x^6)

In [5]:
s = 5
n = 5
tree_s = MAryTrees(s)
tree_s

5-ary trees

Testiamo ora la definizione di albero $s$-ario, utilizzando la funzione *random_element* che estrae una permutazione a caso;

In [6]:
random_s_tree = tree_s._get_m_ary_trees_size(n).random_element()
random_s_tree

[., ., ., [., ., ., ., .], [., [., ., ., [., ., ., ., .], .], ., ., .]]

### Calcoliamo il numero totale di strutture possibili

Costruiamo un albero di arietà 5 e restituiiamo l'insieme degli alberi $s$-ari di dimensione $n = 10$. Dopodichè calcoliamo la dimensione dell'insieme.

In [7]:
concrete_count = len(tree_s._get_m_ary_trees_size(10))
concrete_count

Calcoliamo il valore teorico tramite la formula proposta dalla teoria
$$\frac{1}{(s-1) \cdot n+1}\binom{s\cdot n}{n}$$

Calcolo tramite la funzione del pacchetto MaryTree

In [8]:
theoretical_count = tree_s._get_m_ary_trees_size(n).cardinality()
theoretical_count

2530

Calcolo esplicitando la formulazione teorica

In [9]:
def val_teorico(s, n):
    return 1 / ((s - 1) * n + 1) * binomial(s * n, n)

val_teorico(s, n)

2530

Come si evince i due risultati coincidono.

Proponiamo quindi anche una procedura che, dato un valore $k$ (pari al numero di simulazioni da condurre, ovvero il numero di alberi da creare), restituisce il numero totale di foglie presenti all'interno di tutti i $k$ alberi $s$-ari creati

In [10]:
def count_leaves(tree, leave_structure="[., .]"):
    return ' '.join(map(str, tree)).count(leave_structure)

In [59]:
from functools import lru_cache

def get_leaves_str(size):
    structure = '['
    for _ in range(size - 1):
        structure+='., '
    
    return structure + '.]'
    

def get_random_of(trees):
    rand_index = randint(0, trees.cardinality() - 1)
    for idx, tree in enumerate(trees):
        if idx == rand_index:
            return tree
    return None

@cached_function
def count_leaves_s_tree(s, n, k):
    acc = 0
    leave_struct = get_leaves_str(s)
    tree_s = MAryTrees(s)._get_m_ary_trees_size(n)
    
    for _ in tqdm(range(1, k + 1)):
        tree = get_random_of(tree_s)
        cl = count_leaves(tree, leave_structure=leave_struct)
        acc += cl
        #print(f"internal nodes: {n - cl}, external nodes: {cl}")

    return acc

Richiamiamo la procedura precedente con $k=10000$ (ovvero con $10000$ simulazioni di alberi s-ari) e salviamo il risultato all'interno di una variabile:

In [60]:
ary = 5
num_of_node = 6
iteration = 1
count_leaves_s_tree(ary, num_of_node, iteration)

100%|██████████| 1/1 [00:07<00:00,  7.50s/it]


2

## 2. Alberi Binari
Vogliamo determinare il numero medio di foglie in un albero binario con n nodi interni. Una foglia è un nodo interno in cui entrambi i figli sono nodi esterni. Si può usare il metodo simbolico:
$$\mathcal{B}=\{\bullet\}+\{\circ\}\times \mathcal{B}\times \mathcal{B}$$
Sia $B(t, w)$ la funzione generatrice bivariata che conta il numero $b_{n,k}$ di alberi binari con $n$ nodi interni e $k$ foglie, allora:

$$B(t, w) = 1 + tw + tB(t, w)^2 − t$$

(si sottrae $t$ per evitare che l’albero di misura 1 venga contato 2 volte). Derivando rispetto a $w$ e poi ponendo $w = 1$ si ha: 

$$B_w(t, 1) = t + 2tB(t, 1)B_u(t, 1) = \frac{t}{1 − 2tB(t, 1)}=\frac{t}{\sqrt{1 − 4t}}$$

Quindi, il numero medio di foglie in un albero binario con n nodi interni è:

$$\frac{[t^n]\frac{1}{\sqrt{1−4t}}}{\frac{1}{n+1}\binom{2n}{n}} = \frac{\binom{2n−2}{n−1}}{\frac{1}{n+1}\binom{2n}{n}}=\frac{n(n + 1)}{2(2n − 1)} \approx \frac{n}{4} \quad{\text{ per }} n\rightarrow \infty$$

###  Verifica del numero di foglie negli alberi binari dato il numero di nodi interni.  
Numero di nodi esterni negli alberi binari: $$ \frac{n}{4} \quad \text{per}\quad n \rightarrow \infty.$$


Definiamo la struttura dei suddetti alberi

In [12]:
n = 100
tree_2 = BinaryTrees(n)
tree_2

Binary trees of size 100

Esempio di struttura ad albero Binario contenente 10 nodi

In [13]:
ascii_art(BinaryTrees(10).random_element())

    _____o_____
   /           \
  o             o
   \           /
  __o__       o
 /     \     /
o       o   o
 \     /    
  o   o     

Definiamo ora una procedura che, dati due parametri $k$ e $n$, tramite simulazione ritorna il valore totale delle foglie presenti all'interno di $k$ alberi binari con $n$ nodi interni

In [14]:
def count_leaves_bin(k, n):
    acc = 0
    for i in tqdm(range(k)):
        tree = tree_2.random_element()
        acc += count_leaves(tree)
        
    return acc

Ora richiamiamo la procedura con $k = 10000$ numero di alberi e $n = 100$ numero di nodi interni di ogni albero

In [15]:
k = 10000
n = 100
tot_leaves = count_leaves_bin(k, n)
tot_leaves

100%|██████████| 10000/10000 [00:22<00:00, 436.88it/s]


253697

Ora prendiamo il valore ritornato, lo dividiamo per il numero di alberi su cui abbiamo effettuato la simulazione, e notiamo che il risultato tende a $\frac{n}{4}$

In [16]:
(tot_leaves / k).n()

25.3697000000000

In questo caso $n$ era uguale a $100$, quindi correttamente si ottiene un valore attorno a $25$.

In [17]:
n / 4

25

## 3. Alberi planari con radice
Sia $G(z, u)$ la funzione generatrice bivariata associata a $G_{n,k}$, ovvero:

$$G(z,u)=\sum_{n,k\ge 0}G_{n,k}z^nu^k$$

dove $z$ conta il numero di nodi e $u $quello delle foglie. La classe $\mathbb{A}$ degli alberi aventi nodi di qualsiasi arietà $s$ risulta essere l’unione disgiunta degli insiemi $\mathbb{A}_s$, $s \ge 0$, costituiti dagli alberi di $\mathbb{A}$ che hanno un nodo di arietà $s$ come radice:

$$\mathbb{A} = \{\circ\} \cup \{\circ\} \times \mathbb{A} \cup \{\circ\} \times \mathbb{A} \times \mathbb{A} \cup \{\circ\} \times \mathbb{A} \times \mathbb{A} \times \mathbb{A} \cup \cdots$$

Passando alla funzione generatrice bivariata si ha:

$$G(z,u)=zu+zG(z,u)+zG(z,u)^2+\cdots=zu+z\sum_{k\ge 0}G(z,u)^k=zu+\frac{zG(z,u)}{1-G(z,u)}=z\left(u+\frac{G(z,u)}{1-G(z,u)}\right)$$

Per trovare $G_{n,k}$ la cosa più semplice, piuttosto che risolvere direttamente l'espressione, è applicare la formula di inversione di Lagrange:

$$G_{n,k} = [u^k][z^n]G(z, u) = [u^k]\frac{1}{n}[y^{n-1}]\left(u +\frac{y}{1 - y}\right)^n=\frac{1}{n}\binom{n}{k}[y^{n-1}]\frac{y^{n-k}}{(1 - y)^{n-k}}=\frac{1}{n}\binom{n}{k}\binom{n - 2}{k - 1}.$$

Avendo trovato $G_{n,k}$ in forma esplicita, il numero medio di foglie questa volta può essere determinato calcolando le sequenti somme combinatorie

$$\frac{\sum_{k\ge 0}kG_{n,k}}{\sum_{k\ge 0}G_{n,k}}=\frac{\frac{1}{2}\binom{2n-2}{n-1}}{\frac{1}{n}\binom{2n-2}{n-1}}=\frac{n}{2}$$

### Verifica del numero di foglie negli alberi planari dato il numero di nodi interni.

Il numero di nodi foglia degli alberi planari tende a $\frac{n}{2}$.

In [46]:
o = var('o')
N = SingletonSpecies()
L = EmptySetSpecies()
TreeP = CombinatorialSpecies()
n = 100
tree_def = N + N 

for j in range(1, n + 1):
    tree_def += N * TreeP^j

TreeP.define(tree_def)
TreeP.isotypes([o] * 5).random_element()

KeyboardInterrupt: 

In [50]:
import os
import sys
os.path.dirname(sys.executable)

'/opt/sagemath-9.2/local/bin'

In [22]:
ascii_art(OrderedTrees(10).random_element())

     __o____
    /      /
  _o___   o
 /    /  
o   _o__ 
   / / / 
  o o o  
    | |  
    o o  

Definiamo ora una procedura che, dati due parametri $k$ e $n$, tramite simulazione ritorna il valore totale delle foglie presenti all'interno di $k$ alberi planari con n nodi interni

In [23]:
def count_leaves_planar_tree(k, n):
    acc = 0
    for i in tqdm(range(k)):
        tree = OrderedTrees(n).random_element()
        acc += n - count_leaves(tree, leave_structure="[]")
    
    return acc

Ora richiamiamo la procedura con i valori di $k=10000$ (ovvero $10000$ alberi) e $n=100$ (alberi con 100 nodi interni ciascuno) e salviamo il risultato del totale delle foglie  all'interno di una variabile:

In [24]:
k = 10000
n = 100
tot_leaves_plan = count_leaves_planar_tree(k, n);
tot_leaves_plan

100%|██████████| 10000/10000 [00:11<00:00, 842.19it/s]


500337

In [25]:
(tot_leaves_plan / k).n()

50.0337000000000

In questo caso $n =100$, e quindi il risultato ottenuto di circa $50$ nodi foglia rispecchia il risultato teorico di $\frac{n}{2}$.

In [26]:
n / 2

50

## 4. Alberi binari di ricerca
Un albero binario di ricerca è un albero binario con chiavi associate a ciascun nodo interno con la proprietà che la chiave in un nodo è maggiore di tutte le chiavi nel suo sottoalbero sinistro e minore di tutte quelle nel sottoalbero destro.

- Numero medio di confronti fra chiavi necessari per costruire un albero binario di ricerca inserendo n chiavi distinte in ordine casuale = lunghezza media del cammino totale interno 
$$I_n = n − 1 + \frac{1}{n}\sum^n_{k=1}(I_{k−1} + I_{n−k}),\quad I_0 = 0. \qquad \text{
Si trova} \quad I_n = 2(n + 1)(H_{n+1} − 1) − 2$$

-  Numero medio di confronti in una ricerca con successo: 
$$\frac{1}{n}I_n + 1 = 2H_n − 3 − 2\frac{H_n}{n}$$

-  Numero medio di confronti in una ricerca senza successo: $$\frac{I_n + 2n}{n + 1} = 2H_{n+1} − 2 $$

### Numero medio di foglie negli alberi binari di ricerca dato il numero di nodi interni. 
Il numero medio di foglie in un albero binario di ricerca costruito inserendo n chiavi distinte in ordine casuale soddisfa la ricorrenza:

$$F_n = \delta_{n,1} + \frac{2}{n}\sum^{n−1}_{k=0}F_k,\quad F_0 = 0$$
$\delta_{n,1}$ è la porzione di costo associata alla radice). Passando alle funzione generatrice si trova:

$$tF'(t) = t +\frac{2t}{1 − t}F(t)$$

$$F(t) = \frac{1}{3}\frac{1}{(1 − t)^2}+\frac{1}{3}(t-1)$$
Estraendo il coefficiente si trova infine:
$$F_n =\frac{n + 1}{3}.$$

Il numero medio di foglie all'interno degli alberi binari di ricerca tende a $\frac{n}{3}$.

Siccome gli alberi binari di ricerca non possiedono una struttura definita a priori e regolare come nel caso degli alberi binari standard, o degli alberi $s$, definiamo una procedura che, data una permutazione di m elementi, restituisce una struttura ad albero corrispondente all'albero binario di ricerca costruito su quella specifica permutazione.  Ciò ci permette di calcolare, analogamente a quanto effettuato per le altre analisi, il numero medio di foglie e di confrontarlo con il risultato teorico atteso.

In [27]:
import random 
def random_perm_of_lenght(n):
    arr = list(range(1, n + 1))
    random.shuffle(arr)
    return Permutation(arr)

def permutation_to_bst(perm):
    return perm.binary_search_tree()

Definiamo ora una procedura che, dato un valore $k$ pari al numero di simulazioni da condurre ovvero il numero di alberi da creare e un valore $n$ ovvero il numero di nodi interni, restituisce il numero totale di foglie presenti interno di ogni albero. Tramite questo valore, come per le altre analisi, potremo facilmente calcolare il numero medio di nodi foglia all'interno degli alberi binari di ricerca creati;

In [28]:
def count_abr(k, n):
    acc = 0
    for i in tqdm(range(1, k + 1)):
        bst = permutation_to_bst(random_perm_of_lenght(n))
        acc += count_leaves(bst)
    
    return acc

Richiamiamo la procedura con $k=10000$ (ovvero $10000$ alberi) e $n=100$ (ovvero $100$ nodi interni per ogni albero) e lo assegnamo ad una variabile:

In [29]:
k = 10000
n = 100
tot_leaves_abr = count_abr(k, n)
tot_leaves_abr

100%|██████████| 10000/10000 [01:46<00:00, 94.31it/s]


337066

In [30]:
(tot_leaves_abr / k).n()

33.7066000000000

Come possiamo notare, il risultato ottenuto rispecchia il valore teorico, in quanto si ha che per $n=100$ il numero di nodi foglia dovrebbe risultare circa $33$,  valore ritrovato tramite questa simulazione.

In [31]:
(n / 3).n()

33.3333333333333